# Day18_0: Transformer 어텐션 메커니즘 - 정답

## 📚 실습 퀴즈 정답

---

In [ ]:
# 공통 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

---

### Q1. Attention Score 계산 ⭐

**문제**: Query와 Key가 주어졌을 때, Attention Score를 계산하세요.

In [ ]:
# 정답 코드

# Query와 Key 정의
Q = torch.tensor([[1.0, 0.0], [0.0, 1.0]])  # (2, 2)
K = torch.tensor([[1.0, 1.0], [0.0, 1.0]])  # (2, 2)

# Attention Score 계산: Q @ K^T
scores = torch.matmul(Q, K.transpose(-2, -1))

print("Query Q:")
print(Q)
print("\nKey K:")
print(K)
print("\nAttention Score (Q @ K^T):")
print(scores)
print(f"\nScore shape: {scores.shape}")

### 💡 풀이 설명

**접근 방법**:
- Attention Score는 Query와 Key의 내적(dot product)으로 계산
- Q @ K^T를 통해 각 Query가 각 Key와 얼마나 유사한지 측정

**핵심 개념**:
- Score[i,j] = Q[i] dot K[j]
- 높은 점수 = Query와 Key가 비슷한 방향

**계산 과정**:
```
Q[0] = [1, 0], K[0] = [1, 1] -> 1*1 + 0*1 = 1
Q[0] = [1, 0], K[1] = [0, 1] -> 1*0 + 0*1 = 0
Q[1] = [0, 1], K[0] = [1, 1] -> 0*1 + 1*1 = 1
Q[1] = [0, 1], K[1] = [0, 1] -> 0*0 + 1*1 = 1
```

**실무 팁**:
- transpose(-2, -1)은 마지막 두 차원을 교환
- 배치 처리 시에도 동일하게 작동

---

### Q2. Scaled Dot-Product Attention ⭐

**문제**: Score를 스케일링하고 softmax를 적용하여 어텐션 가중치를 계산하세요.

In [ ]:
# 정답 코드

# 이전 문제의 Score 사용
Q = torch.tensor([[1.0, 0.0], [0.0, 1.0]])
K = torch.tensor([[1.0, 1.0], [0.0, 1.0]])
scores = torch.matmul(Q, K.transpose(-2, -1))

# Key 차원
d_k = K.size(-1)  # 2

# 스케일링: Score / sqrt(d_k)
scaled_scores = scores / math.sqrt(d_k)

# Softmax로 가중치 계산
attention_weights = F.softmax(scaled_scores, dim=-1)

print(f"Score:\n{scores}")
print(f"\nd_k = {d_k}, sqrt(d_k) = {math.sqrt(d_k):.4f}")
print(f"\nScaled Score:\n{scaled_scores}")
print(f"\nAttention Weights (softmax):\n{attention_weights}")
print(f"\n각 행의 합: {attention_weights.sum(dim=-1)}")

### 💡 풀이 설명

**접근 방법**:
1. Score를 sqrt(d_k)로 나눠 스케일링
2. Softmax를 적용하여 확률 분포 생성

**핵심 개념**:
- 스케일링: 값이 너무 크면 softmax가 극단적인 값을 출력
- sqrt(d_k)로 나눠 분산을 1로 유지
- Softmax: 각 행의 합이 1이 되도록 정규화

**왜 스케일링?**:
- d_k가 크면 내적 값이 커짐 (분산 ~ d_k)
- 큰 값 -> softmax 후 0 또는 1 -> 기울기 소실
- sqrt(d_k)로 나눠 분산 안정화

**실무 팁**:
- dim=-1: 마지막 차원(Key 방향)에 softmax 적용
- 각 Query가 모든 Key에 대해 합이 1인 가중치 생성

---

### Q3. Query, Key, Value 생성 ⭐⭐

**문제**: 입력 텐서 X로부터 Q, K, V를 생성하는 선형 레이어를 정의하고 적용하세요.

In [ ]:
# 정답 코드

# 설정
batch_size = 1
seq_len = 4
embed_dim = 8

# 입력 텐서
X = torch.randn(batch_size, seq_len, embed_dim)

# Q, K, V 생성을 위한 선형 레이어
W_Q = nn.Linear(embed_dim, embed_dim, bias=False)
W_K = nn.Linear(embed_dim, embed_dim, bias=False)
W_V = nn.Linear(embed_dim, embed_dim, bias=False)

# Q, K, V 생성
Q = W_Q(X)
K = W_K(X)
V = W_V(X)

print(f"입력 X shape: {X.shape}")
print(f"Query Q shape: {Q.shape}")
print(f"Key K shape: {K.shape}")
print(f"Value V shape: {V.shape}")
print(f"\nW_Q 파라미터 shape: {W_Q.weight.shape}")

### 💡 풀이 설명

**접근 방법**:
- nn.Linear로 Q, K, V 변환 행렬 정의
- 각 행렬은 동일한 입력 X를 다른 방식으로 변환

**핵심 개념**:
- Q = X @ W_Q: "내가 찾는 정보" 표현
- K = X @ W_K: "내가 가진 정보의 특성" 표현
- V = X @ W_V: "실제 정보 내용" 표현

**왜 세 개로 분리?**:
- 같은 입력이지만 다른 "역할"을 수행
- Q-K 내적 = 유사도 계산
- V = 실제 전달될 정보

**실무 팁**:
- bias=False: 원 논문에서는 bias 사용, 하지만 없어도 성능 유사
- 효율성을 위해 Q, K, V를 한 번에 계산하기도 함

---

### Q4. Self-Attention 전체 계산 ⭐⭐

**문제**: Q, K, V가 주어졌을 때, Self-Attention의 전체 계산을 수행하세요.

In [ ]:
# 정답 코드

# Q, K, V 생성 (이전 문제에서)
batch_size, seq_len, embed_dim = 1, 4, 8
X = torch.randn(batch_size, seq_len, embed_dim)

W_Q = nn.Linear(embed_dim, embed_dim, bias=False)
W_K = nn.Linear(embed_dim, embed_dim, bias=False)
W_V = nn.Linear(embed_dim, embed_dim, bias=False)

Q = W_Q(X)
K = W_K(X)
V = W_V(X)

d_k = Q.size(-1)

# 1. Score 계산: Q @ K^T
scores = torch.matmul(Q, K.transpose(-2, -1))
print(f"1. Score shape: {scores.shape}")

# 2. 스케일링: / sqrt(d_k)
scaled_scores = scores / math.sqrt(d_k)
print(f"2. Scaled Score shape: {scaled_scores.shape}")

# 3. Softmax
attention_weights = F.softmax(scaled_scores, dim=-1)
print(f"3. Attention Weights shape: {attention_weights.shape}")

# 4. 가중합: weights @ V
output = torch.matmul(attention_weights, V)
print(f"4. Output shape: {output.shape}")

print(f"\n입력 X와 출력 shape 비교: {X.shape} -> {output.shape}")

### 💡 풀이 설명

**접근 방법**:
Scaled Dot-Product Attention 공식 단계별 적용:
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

**핵심 개념**:
1. Score = Q @ K^T: (B, N, D) @ (B, D, N) = (B, N, N)
2. 스케일링: 값 안정화
3. Softmax: 확률 분포 생성 (각 행 합 = 1)
4. 가중합: (B, N, N) @ (B, N, D) = (B, N, D)

**결과 해석**:
- 출력의 각 위치는 모든 위치의 Value를 가중 합산
- 관련 있는 위치의 정보가 더 많이 반영됨

**실무 팁**:
- 입력과 출력의 shape가 동일 (B, N, D)
- 이를 통해 레이어 쌓기 가능

---

### Q5. 어텐션 가중치 시각화 ⭐⭐

**문제**: 5개 단어로 구성된 문장의 Self-Attention 가중치를 히트맵으로 시각화하세요.

In [ ]:
# 정답 코드

# 단어 리스트
words = ["I", "love", "deep", "learning", "!"]

# Self-Attention 클래스 정의
class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.embed_dim = embed_dim
    
    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.embed_dim)
        weights = F.softmax(scores, dim=-1)
        output = torch.matmul(weights, V)
        
        return output, weights

# 모델 및 입력 생성
embed_dim = 32
attention = SelfAttention(embed_dim)
x = torch.randn(1, len(words), embed_dim)

# 어텐션 계산
_, attention_weights = attention(x)
attn_matrix = attention_weights.squeeze().detach().numpy()

# Plotly 히트맵 시각화
fig = px.imshow(
    attn_matrix,
    labels=dict(x="Key (참조되는 단어)", y="Query (주목하는 단어)", color="가중치"),
    x=words,
    y=words,
    color_continuous_scale="Blues",
    aspect="equal"
)

fig.update_layout(
    title="Self-Attention 가중치 히트맵",
    xaxis_title="Key (참조되는 단어)",
    yaxis_title="Query (주목하는 단어)"
)

fig.show()

print("어텐션 가중치 행렬:")
print(pd.DataFrame(attn_matrix, index=words, columns=words).round(3))

### 💡 풀이 설명

**접근 방법**:
1. Self-Attention 클래스 정의
2. 어텐션 가중치 추출
3. px.imshow로 히트맵 시각화

**핵심 개념**:
- 행(Query): 주목하는 단어
- 열(Key): 참조되는 단어
- 값: 주목 정도 (0~1, 행 합=1)

**해석 방법**:
- 밝은 색: 높은 어텐션 (강한 연관성)
- 대각선: 자기 자신에 대한 어텐션
- 비대각선: 다른 단어와의 관계

**실무 팁**:
- 실제 모델에서는 학습 후 의미 있는 패턴 나타남
- 초기화 직후는 랜덤에 가까움

---

### Q6. Multi-Head Attention 헤드 분할 ⭐⭐⭐

**문제**: embed_dim=64, num_heads=4일 때, 텐서를 헤드별로 분할하세요.

In [ ]:
# 정답 코드

# 설정
batch_size = 2
seq_len = 10
embed_dim = 64
num_heads = 4
head_dim = embed_dim // num_heads  # 64 / 4 = 16

# 입력 텐서 (예: Q, K, 또는 V)
x = torch.randn(batch_size, seq_len, embed_dim)
print(f"원본 shape: {x.shape}")
print(f"(batch={batch_size}, seq_len={seq_len}, embed_dim={embed_dim})")

# 헤드별 분할
# Step 1: (B, N, D) -> (B, N, h, d_k)
x_reshaped = x.view(batch_size, seq_len, num_heads, head_dim)
print(f"\nreshape 후: {x_reshaped.shape}")

# Step 2: (B, N, h, d_k) -> (B, h, N, d_k)
x_heads = x_reshaped.transpose(1, 2)
print(f"transpose 후: {x_heads.shape}")
print(f"(batch={batch_size}, num_heads={num_heads}, seq_len={seq_len}, head_dim={head_dim})")

### 💡 풀이 설명

**접근 방법**:
1. view로 임베딩 차원을 (num_heads, head_dim)으로 분할
2. transpose로 헤드 차원을 seq_len 앞으로 이동

**핵심 개념**:
- 각 헤드가 독립적으로 어텐션 수행
- head_dim = embed_dim / num_heads
- (B, N, D) -> (B, h, N, d_k)

**왜 transpose?**:
- 어텐션 계산에서 (N, d_k) 형태 필요
- Q @ K^T 계산 시 마지막 두 차원 사용
- 배치 병렬 처리 효율적

**실무 팁**:
- embed_dim % num_heads == 0 필수
- contiguous() 호출 필요할 수 있음 (메모리 연속성)

---

### Q7. Positional Encoding 생성 ⭐⭐⭐

**문제**: d_model=16, max_len=10인 Positional Encoding을 생성하고 시각화하세요.

In [ ]:
# 정답 코드

# 설정
d_model = 16
max_len = 10

# Positional Encoding 생성
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

# 짝수 인덱스: sin, 홀수 인덱스: cos
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

print(f"Positional Encoding shape: {pe.shape}")
print(f"(max_len={max_len}, d_model={d_model})")

# 히트맵 시각화
fig = px.imshow(
    pe.numpy(),
    labels=dict(x="Dimension", y="Position", color="Value"),
    color_continuous_scale="RdBu",
    aspect="auto"
)

fig.update_layout(
    title=f"Positional Encoding (d_model={d_model}, max_len={max_len})",
    xaxis_title="Dimension",
    yaxis_title="Position"
)

fig.show()

In [ ]:
# 특정 차원의 패턴 시각화
positions = list(range(max_len))

fig = go.Figure()

# 처음 4개 차원 시각화
for dim in range(4):
    fig.add_trace(go.Scatter(
        x=positions,
        y=pe[:, dim].numpy(),
        mode='lines+markers',
        name=f'dim {dim} ({"sin" if dim % 2 == 0 else "cos"})'
    ))

fig.update_layout(
    title="Positional Encoding: 각 차원별 패턴",
    xaxis_title="Position",
    yaxis_title="Value"
)

fig.show()

### 💡 풀이 설명

**접근 방법**:
1. PE 행렬 초기화 (max_len x d_model)
2. 각 위치에 대해 sin/cos 값 계산
3. 짝수/홀수 인덱스에 각각 할당

**핵심 수식**:
$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

**패턴 해석**:
- 낮은 차원: 빠르게 진동 (고주파)
- 높은 차원: 천천히 진동 (저주파)
- 각 위치마다 고유한 패턴 생성

**왜 sin/cos?**:
- 주기성: 상대적 위치 관계 학습 가능
- 외삽 가능: 학습 시 본 적 없는 긴 시퀀스에도 적용
- PE(pos+k)를 PE(pos)의 선형 결합으로 표현 가능

---

### Q8. Transformer Encoder Layer 구현 ⭐⭐⭐⭐

**문제**: Multi-Head Attention, Add & Norm, Feed-Forward를 포함하는 Encoder Layer를 구현하세요.

In [ ]:
# 정답 코드

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.fc_out = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        B, N, _ = x.shape
        
        Q = self.query(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        weights = F.softmax(scores, dim=-1)
        context = torch.matmul(weights, V)
        
        context = context.transpose(1, 2).contiguous().view(B, N, self.embed_dim)
        return self.fc_out(context)


class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        
        # Multi-Head Attention
        self.attention = MultiHeadAttention(embed_dim, num_heads)
        
        # Layer Normalization
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        # Feed-Forward Network
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, embed_dim)
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # 1. Multi-Head Attention + Add & Norm
        attn_output = self.attention(x)
        x = self.norm1(x + self.dropout(attn_output))
        
        # 2. Feed-Forward + Add & Norm
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x


# 테스트
embed_dim = 64
num_heads = 4
ff_dim = 256

encoder_layer = TransformerEncoderLayer(embed_dim, num_heads, ff_dim)
x = torch.randn(2, 10, embed_dim)  # (batch, seq_len, embed_dim)
output = encoder_layer(x)

print(f"입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")
print(f"\n파라미터 수: {sum(p.numel() for p in encoder_layer.parameters()):,}")

### 💡 풀이 설명

**접근 방법**:
1. MultiHeadAttention 클래스 정의
2. TransformerEncoderLayer에서 조립
3. Add & Norm (잔차 연결 + LayerNorm) 적용

**핵심 구조**:
```
x -> [Multi-Head Attention] -> [Add & Norm] -> [Feed-Forward] -> [Add & Norm] -> output
```

**핵심 개념**:
- Add & Norm: x + dropout(sublayer(x)) 후 LayerNorm
- 잔차 연결: 기울기 흐름 개선, 깊은 모델 학습 가능
- LayerNorm: 학습 안정화

**실무 팁**:
- Pre-LN vs Post-LN: 최근에는 LayerNorm을 sublayer 전에 적용하는 Pre-LN이 더 안정적
- ff_dim은 보통 embed_dim의 4배

---

### Q9. nn.TransformerEncoder 활용 ⭐⭐⭐⭐

**문제**: PyTorch의 nn.TransformerEncoder를 사용하여 6개 레이어의 Encoder를 만들고 순전파를 수행하세요.

In [ ]:
# 정답 코드

# 설정
d_model = 256
nhead = 8
dim_feedforward = 1024
num_layers = 6
dropout = 0.1

# Encoder Layer 정의
encoder_layer = nn.TransformerEncoderLayer(
    d_model=d_model,
    nhead=nhead,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
    batch_first=True  # (batch, seq, feature) 형식 사용
)

# Encoder (6 layers)
encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

# 순전파 테스트
batch_size = 4
seq_len = 50
x = torch.randn(batch_size, seq_len, d_model)

output = encoder(x)

print(f"nn.TransformerEncoder 설정:")
print(f"  - d_model: {d_model}")
print(f"  - nhead: {nhead}")
print(f"  - dim_feedforward: {dim_feedforward}")
print(f"  - num_layers: {num_layers}")
print(f"\n입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")
print(f"\n총 파라미터: {sum(p.numel() for p in encoder.parameters()):,}")

### 💡 풀이 설명

**접근 방법**:
1. nn.TransformerEncoderLayer로 단일 레이어 정의
2. nn.TransformerEncoder로 여러 레이어 스택
3. batch_first=True로 (batch, seq, feature) 형식 사용

**핵심 파라미터**:
- d_model: 임베딩/모델 차원
- nhead: Multi-Head Attention 헤드 수
- dim_feedforward: Feed-Forward 내부 차원
- num_layers: Encoder 레이어 수

**원 논문 설정**:
- Base: d_model=512, nhead=8, ff=2048, layers=6
- Big: d_model=1024, nhead=16, ff=4096, layers=6

**실무 팁**:
- batch_first=True: PyTorch 1.9+에서 지원
- Mask 사용: src_mask (패딩), src_key_padding_mask (어텐션 마스크)

---

### Q10. Transformer 텍스트 분류 모델 ⭐⭐⭐⭐⭐

**문제**: Transformer 기반 텍스트 분류 모델을 구현하고, 가상의 데이터로 1 에포크 학습을 수행하세요.

In [ ]:
# 정답 코드 - Part 1: Positional Encoding 및 모델 정의

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


class TransformerTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, num_layers, num_classes, max_len=512, dropout=0.1):
        super().__init__()
        
        # 1. Embedding + Positional Encoding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len, dropout)
        
        # 2. Transformer Encoder (2 layers)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 3. 분류 헤드 (평균 풀링 + FC)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim // 2, num_classes)
        )
    
    def forward(self, x):
        # x: (batch, seq_len) - 토큰 인덱스
        x = self.embedding(x)       # (batch, seq_len, embed_dim)
        x = self.pos_encoding(x)    # + positional encoding
        x = self.encoder(x)         # Transformer Encoder
        
        # 평균 풀링
        x = x.mean(dim=1)           # (batch, embed_dim)
        x = self.classifier(x)      # (batch, num_classes)
        
        return x


print("모델 클래스 정의 완료!")

In [ ]:
# 정답 코드 - Part 2: 데이터 준비 및 모델 생성

# 설정
vocab_size = 10000
embed_dim = 128
num_heads = 4
ff_dim = 256
num_layers = 2
num_classes = 2
max_len = 100
batch_size = 32

# 가상의 데이터 생성 (실제로는 토큰화된 텍스트 사용)
num_samples = 500
X_train = torch.randint(0, vocab_size, (num_samples, max_len))
y_train = torch.randint(0, num_classes, (num_samples,))

# DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 모델 생성
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerTextClassifier(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_dim=ff_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    max_len=max_len
).to(device)

# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Device: {device}")
print(f"학습 데이터: {len(train_dataset)} 샘플")
print(f"배치 수: {len(train_loader)}")
print(f"모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 정답 코드 - Part 3: 학습 루프 (1 에포크)

model.train()
total_loss = 0
correct = 0
total = 0
losses = []

print("=" * 60)
print("학습 시작! (1 에포크)")
print("=" * 60)

for batch_idx, (texts, labels) in enumerate(train_loader):
    texts, labels = texts.to(device), labels.to(device)
    
    # Forward
    optimizer.zero_grad()
    outputs = model(texts)
    loss = criterion(outputs, labels)
    
    # Backward
    loss.backward()
    optimizer.step()
    
    # 통계
    total_loss += loss.item()
    losses.append(loss.item())
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    
    if batch_idx % 5 == 0:
        print(f"Batch {batch_idx:3d}/{len(train_loader)}: Loss={loss.item():.4f}")

avg_loss = total_loss / len(train_loader)
accuracy = 100. * correct / total

print("=" * 60)
print(f"에포크 완료!")
print(f"  평균 Loss: {avg_loss:.4f}")
print(f"  정확도: {accuracy:.2f}%")
print("=" * 60)

In [ ]:
# 정답 코드 - Part 4: 학습 곡선 시각화

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, len(losses) + 1)),
    y=losses,
    mode='lines',
    name='Loss'
))

fig.update_layout(
    title='학습 손실 (1 에포크)',
    xaxis_title='Batch',
    yaxis_title='Loss'
)

fig.show()

### 💡 풀이 설명

**전체 파이프라인**:
1. **PositionalEncoding**: sin/cos 패턴으로 위치 정보 생성
2. **TransformerTextClassifier**:
   - Embedding -> Positional Encoding -> TransformerEncoder -> Mean Pooling -> Classifier
3. **학습 루프**: Forward -> Loss -> Backward -> Update

**모델 구조**:
```
토큰 인덱스 (batch, seq_len)
    ↓ Embedding
임베딩 (batch, seq_len, embed_dim)
    ↓ Positional Encoding
위치 정보 추가된 임베딩
    ↓ TransformerEncoder (2 layers)
문맥 벡터 (batch, seq_len, embed_dim)
    ↓ Mean Pooling
문장 벡터 (batch, embed_dim)
    ↓ Classifier (FC)
클래스 점수 (batch, num_classes)
```

**핵심 개념**:
- Mean Pooling: 모든 토큰의 평균으로 문장 표현
- 대안: [CLS] 토큰 사용 (BERT 스타일)

**하이퍼파라미터 가이드**:
| 설정 | 작은 모델 | 중간 모델 | 큰 모델 |
|-----|---------|---------|--------|
| embed_dim | 128 | 256 | 512 |
| num_heads | 4 | 8 | 8 |
| ff_dim | 256 | 1024 | 2048 |
| num_layers | 2 | 4 | 6 |

**실무 팁**:
- 실제 학습 시 수십 에포크 필요
- 학습률 스케줄러 (warmup) 권장
- 배치 크기는 메모리 허용 범위 내 최대로

---

## 📊 학습 정리

### Transformer 핵심 요약

| 개념 | 핵심 수식/코드 | 역할 |
|-----|---------------|------|
| Self-Attention | softmax(QK^T/sqrt(d_k))V | 시퀀스 내 관계 파악 |
| Multi-Head | Concat(head_1, ..., head_h)W_O | 다양한 관점 통합 |
| Positional Encoding | sin/cos 패턴 | 순서 정보 주입 |
| Add & Norm | x + sublayer(x), LayerNorm | 학습 안정화 |
| Feed-Forward | Linear -> ReLU -> Linear | 비선형 변환 |

### Attention 공식

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### 실무 체크리스트

- [ ] embed_dim % num_heads == 0 확인
- [ ] Positional Encoding 적용
- [ ] batch_first=True 설정 (PyTorch 1.9+)
- [ ] Dropout 적절히 사용 (0.1~0.3)
- [ ] 학습률 warmup 고려
- [ ] 긴 시퀀스 시 메모리 주의 (O(n^2))